# Generate square grids

In [1]:
import time
import os
import base64
import hashlib
import geopandas as gpd
import requests
import re
from dotenv import load_dotenv
from shapely.geometry import Polygon, MultiPolygon, GeometryCollection, Point

C:\Users\ajimp\AppData\Roaming\Python\Python38\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-1-93708b63e478>:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migratio

In [3]:
def create_grid_polygons(gdf, bounds, square_cell_length, num_rows, num_cols):
    # Create a list of polygons representing the grid and calculate their areas and centroids
    polygons = []
    areas = []
    centroids = []
    for i in range(num_rows):
        for j in range(num_cols):
            # Calculate the coordinates of the vertices of the grid cell
            minx = bounds[0] + j * square_cell_length
            maxx = minx + square_cell_length
            miny = bounds[1] + i * square_cell_length
            maxy = miny + square_cell_length
            vertices = [(minx, miny), (maxx, miny), (maxx, maxy), (minx, maxy)]

            # Create a polygon from the vertices and calculate its intersection with the input shapefile
            grid_cell = Polygon(vertices)
            intersection = grid_cell.intersection(gdf.unary_union)

            if intersection.is_empty:
                # If the intersection is empty, skip the grid cell
                continue

            if isinstance(intersection, Polygon):
                # If the intersection is a Polygon, add it to the list of polygons and calculate its area and centroid
                polygons.append(intersection)
                areas.append(intersection.area)
                centroids.append(intersection.centroid)
            elif isinstance(intersection, (MultiPolygon, GeometryCollection)):
                # If the intersection is a MultiPolygon or GeometryCollection, add each Polygon to the list of polygons and calculate its area and centroid
                for geom in intersection.geoms:
                    if isinstance(geom, Polygon):
                        polygons.append(geom)
                        areas.append(geom.area)
                        centroids.append(geom.centroid)

    # Add print statements to see what's going on
    print(f"num_rows={num_rows}, num_cols={num_cols}")
    print(f"bounds={bounds}")
    print(f"square_cell_length={square_cell_length}")
    print(f"len(polygons)={len(polygons)}")
    print(f"len(areas)={len(areas)}")
    print(f"len(centroids)={len(centroids)}")

    return polygons, areas, centroids

In [4]:
def create_grid_gdf(polygons, areas, centroids, crs):
    # Create a GeoDataFrame from the list of polygons, areas, and centroids, and the same CRS as the input data
    grid_gdf = gpd.GeoDataFrame(geometry=polygons, crs=crs)
    grid_gdf['area'] = areas
    grid_gdf['centroid'] = [centroid.wkt for centroid in centroids]

    return grid_gdf

In [5]:
def spatial_join(target_gdf, join_gdf, join_columns, target_crs=None):
    """
    Performs a spatial join between a target and a join shapefile based on their geometry, and saves the output to a new shapefile.
    
    Parameters:
    target_file (str): File path of the target shapefile.
    join_file (str): File path of the join shapefile.
    join_columns (list of str): List of column names from the join shapefile to add to the output. This list must
        include the 'geometry' column.
    output_file (str): File path of the output shapefile.
    target_crs (str or dict): The CRS of the target shapefile. If not provided, the function will attempt to
        read the CRS from the shapefile. The CRS can be specified as an EPSG code (e.g., 'EPSG:4326') or as a
        dictionary of proj4 parameters (e.g., {'proj': 'longlat', 'ellps': 'WGS84', 'datum': 'WGS84'}).
    
    Returns:
    None
    """

    # Reproject the target shapefile if needed
    if target_crs and target_gdf.crs != target_crs:
        target_gdf = target_gdf.to_crs(target_crs)

    # Reproject the join shapefile if needed
    if join_gdf.crs != target_gdf.crs:
        join_gdf = join_gdf.to_crs(target_gdf.crs)

    # Make sure the 'geometry' column is included in the join columns
    if 'geometry' not in join_columns:
        join_columns.append('geometry')

    # Perform the spatial join operation
    joined_gdf = gpd.sjoin(
        target_gdf, join_gdf[join_columns], how='left', predicate='intersects')

    # Project the joined GeoDataFrame to EPSG:2263 before computing the area of the intersection
    joined_gdf = joined_gdf.to_crs('EPSG:2263')

    # Compute the area of the intersection between each grid and each borough
    joined_gdf['int_area'] = joined_gdf.geometry.area

    # Group the joined GeoDataFrame by grid ID and select the row with the highest intersection area
    max_intersection_idxs = joined_gdf.groupby(
        'grid_id')['int_area'].idxmax()

    # Select the rows with the highest intersection area and dissolve the resulting GeoDataFrame
    joined_gdf = joined_gdf.loc[max_intersection_idxs].dissolve(by='grid_id')

    # return the new shapefile
    return joined_gdf


In [11]:
def create_grid(shapefile_path, square_cell_length, output_folder):
    # Load the shapefile into a GeoDataFrame
    gdf = gpd.read_file(shapefile_path)

    # Reproject to a suitable projected CRS using UTM zones
    utm_zone = int((gdf.bounds.minx.mean() + 180) / 6) + 1
    projected_crs = f'+proj=utm +zone={utm_zone} +datum=WGS84 +units=m +no_defs'
    gdf = gdf.to_crs(projected_crs)

    # Calculate the bounds of the shapefile
    bounds = gdf.total_bounds

    # Calculate the number of rows and columns of the grid
    num_cols = int((bounds[2] - bounds[0]) / square_cell_length) + 1
    num_rows = int((bounds[3] - bounds[1]) / square_cell_length) + 1

    # Create a list of polygons representing the grid and calculate their areas and centroids
    polygons, areas, centroids = create_grid_polygons(
        gdf, bounds, square_cell_length, num_rows, num_cols)

    # Create a GeoDataFrame from the list of polygons, areas, and centroids, and the same CRS as the input data
    grid_gdf = create_grid_gdf(polygons, areas, centroids, projected_crs)

    # Use the SHA-256 hashing algorithm to generate unique IDs based on the grid's geometries
    id_col = grid_gdf.geometry.apply(lambda geom: base64.urlsafe_b64encode(
        hashlib.sha256(geom.wkb).digest()[:8]).decode('utf-8'))
    grid_gdf['grid_id'] = id_col

    # Spatial join between the 'grid' and 'borough' geodataframes
    grid_gdf = spatial_join(grid_gdf, gdf,
                            ['CODEMAMROT', 'NOM', 'TYPE', 'ABREV'], 'EPSG:4326')

    # Reproject the GeoDataFrame back to the original CRS
    grid_gdf = grid_gdf.to_crs(gdf.crs)

    # Create the output file path
    output_file = os.path.join(
        output_folder, f'square_grids{square_cell_length}.shp')

    # Remove any existing file with the same name
    if os.path.exists(output_file):
        os.remove(output_file)

    # Save the new GeoDataFrame as a shapefile
    grid_gdf.to_file(output_file)

# Generate different grip-maps

In [13]:
# Generate grids with 250 x 250 dimensions
create_grid('Data/Boroughs/LIMADMIN.shp',
            250, 'Data/Generated_grids/')


num_rows=145, num_cols=162
bounds=[ 578451.10752708 5026439.79020024  618798.3836136  5062688.65392742]
square_cell_length=250
len(polygons)=10246
len(areas)=10246
len(centroids)=10246


<ipython-input-11-c12840aa7ec3>:45: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grid_gdf.to_file(output_file)


In [14]:
# Generate grids with 500 x 500 dimensions
create_grid('Data/Boroughs/LIMADMIN.shp',
            500, 'Data/Generated_grids/')

num_rows=73, num_cols=81
bounds=[ 578451.10752708 5026439.79020024  618798.3836136  5062688.65392742]
square_cell_length=500
len(polygons)=2662
len(areas)=2662
len(centroids)=2662


<ipython-input-11-c12840aa7ec3>:45: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grid_gdf.to_file(output_file)


In [15]:
# Generate grids with 750 x 750 dimensions
create_grid('Data/Boroughs/LIMADMIN.shp',
            750, 'Data/Generated_grids/')

num_rows=49, num_cols=54
bounds=[ 578451.10752708 5026439.79020024  618798.3836136  5062688.65392742]
square_cell_length=750
len(polygons)=1211
len(areas)=1211
len(centroids)=1211


<ipython-input-11-c12840aa7ec3>:45: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grid_gdf.to_file(output_file)


In [16]:
# Generate grids with 1000 x 1000 dimensions
create_grid('Data/Boroughs/LIMADMIN.shp',
            1000, 'Data/Generated_grids/')

num_rows=37, num_cols=41
bounds=[ 578451.10752708 5026439.79020024  618798.3836136  5062688.65392742]
square_cell_length=1000
len(polygons)=714
len(areas)=714
len(centroids)=714


<ipython-input-11-c12840aa7ec3>:45: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grid_gdf.to_file(output_file)


# Function to associate 'Lat/Lon' to grid ID

In [19]:
def get_grid_id(lat, lon, grid_file_path):
    # Load the grid file into a GeoDataFrame
    grid_gdf = gpd.read_file(grid_file_path)

    # Assign an appropriate CRS for lat/lon coordinates to the GeoDataFrame
    grid_gdf = grid_gdf.to_crs("EPSG:4326")

    # Convert the input lat/lon coordinates to a Point geometry
    point = Point(lon, lat)

    # Check if the point is within any of the grid polygons
    intersecting_gdf = grid_gdf[grid_gdf.intersects(point)]

    # If there are no intersecting grids, return None
    if intersecting_gdf.empty:
        return None

    # Otherwise, return the ID of the first intersecting grid
    return intersecting_gdf.iloc[0]['grid_id']

In [21]:
# Example return 'grid_id' in the 'square_grids1000.shp' using the McGill University coordinates
grid_id = get_grid_id(45.504785, -73.5747,
                      'Data/Generated_grids/square_grids1000.shp')
print(grid_id)

-niBZlFF5cM=


# Function to convert street names into grid ID

In [23]:
!pip install python-dotenv

In [24]:
load_dotenv()  # Load environment variables from .env file

True

In [25]:
def get_lat_long(address):
    api_key = os.getenv('GOOGLE_MAPS_API_KEY')
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}'
    response = requests.get(url)
    json_response = response.json()
    lat = json_response['results'][0]['geometry']['location']['lat']
    lng = json_response['results'][0]['geometry']['location']['lng']
    return lat, lng

In [26]:
# Example return lat, lng for '1771 rue Leprohon (MTL)'
lat, lng = get_lat_long('1771 rue Leprohon (MTL)')
print(lat, lng)


45.4602905 -73.5876605


In [27]:
# Example to get 'grid_id' of the following coordinates (45.4602905 -73.5876605)
grid_id = get_grid_id(45.4602905, -73.5876605,
                      'Data/Generated_grids/square_grids1000.shp')
print(grid_id)

BVIB7ELQrA4=


# Function to extract square length from file

In [2]:
def extract_number_from_filename(filename):
    number = re.search(r'\d+', filename).group(0)
    return str(number)

In [3]:
# Example of extracting 'square length' from shapefile name
filename = 'square_grids750.shp'
number_str = extract_number_from_filename(filename)
print(number_str)

750
